In [ ]:
%load_ext rpy2.ipython


# DS Helper lecture: Statistics in R    

7th Apr 2021

Lecturers: Alvaro, Jing
---

# Chapter I

Ok,good afternoon, everyone. Welcome to the 7th DS session which,fortunately, is held on 7th April. No wonder 7 is a lucky number. 

Some of you may wonder, why this topic sounds so familiar or did I see it somewhere else? Yes, this is actually a derivative work from Valentina's previous lecture?2701_Intro_statistics _bioinformatics which gives us both a well-rounded reference and various inspirations during the session preparation process. I would like to firstly say "thank you" to our diligent and great organizor: Valentina.

Also, thank you for your precious and valuable feedbacks on the survey that greatly contribute to our growth. According to your feedbacks, we realized improvements we can make and we couldn't thank you more for your precious suggestions.

Alright, let's back on our main track.
Firstly, I want to make sure that everyone's computer is ready for our lecture. Run this check to see if you can see the feedback like this:

In [ ]:
%%R
library('tidyverse')
sessionInfo()

R, by comparison to other programming languages, is well-known for its powerful statistical functions. So today we are gonna check this claim out with one our of friends: movie data set


In [ ]:
#%%R
path_to_file = "https://raw.githubusercontent.com/almenal/imdb_data/main/movies.csv"
movies = readr::read_csv(path_to_file)
movies

##Mean value
Mean value is a central value of a finite set of numbers: specifically, the sum of the values divided by the number of values. In R, we use `mean()` function.


In [ ]:
#%%R
a = mean(movies$budget)
print(a)

##Code with me: This time we are trying to use some of codes that we learned before

In [ ]:
%%R
#Have a look at the mean value for USA movies

##Standard deviation
Standard deviation measures the differences within a dataset. In R, we use `sd()` function.


In [ ]:
#%%R
a1 = sd(movies$budget)
print(a1)
?sd

##Other functions that occassionaly used. `var()`,`min()`,`max()`,`median()`

In [ ]:
%%R

a2 = var(movies$budget)
print(a2)

a3 = min(movies$budget)
print(a3)

a4 = max(movies$budget)
print(a4)

a5 = median(movies$budget)
print(a5)

a6 = range(movies$budget)
print(a6)
?range

a7 = quantile(movies$budget, probs = seq(0,1,0.25))
print(a7)
?quantile


# Chapter II

Now that we have covered descriptive statistics, which help you understand how your data looks like, it is time to apply that knowledge for hypothesis testing.

## t-test and its variants

Let's start our demonstration of statistical tools with an old friend, the Student's t test.
The idea behind is to compare the mean of two groups and assess if they are significantly different with some measure of certainty.
As you may know, the way that statistics deals with the concept of "significance" is by measuring the "rareness" of the event.
An event will be unique if it happens only **rarely** compared to the control. So in order to assess how rare an event is, we need to know what the baseline (the control) looks like, in order to know where our case falls within that baseline. As an example, if we are measuring the blood pressure of patients before and after taking a drug that reduces it.

In order words, we need to know the distribution of our variable __under the null hypothesis__.


In [ ]:
%%R
## Don't worry too much about this part of code, it's not the focus of the lecture :)
uk_scores = movies$score[movies$country=="UK"]
us_scores = movies$score[movies$country=="USA"]
group_means = tibble(country=c("UK","USA"), grpmean=c(mean(uk_scores), mean(us_scores)))

ggplot(movies, aes(x=score, color=country, fill = country)) +
  geom_density(data=movies %>% filter(country == "USA"),alpha=0.4) +
  geom_density(data=movies %>% filter(country == "UK"),alpha=0.4) +
  geom_vline(data = group_means, aes(xintercept = grpmean, color=country), size=1, linetype=2) + 
  annotate("segment", x=min(group_means$grpmean), xend=max(group_means$grpmean), y=0.55, yend = 0.55, color="black") + 
  labs(title = "Visual representation of the t-test procedure",
       subtitle = "The t-test gives evaluates whether the difference in the means (-- lines) is significantly different from zero.\nFor that, it computes the probability of observing a difference like this one under the assumption\nthat there is no difference (i.e. observing this difference by chance)")


In order to run a t-test, we use the function `t.test()`, which takes as input two vectors (the vector for scores of UK movies, and the one for USA movies).


In [ ]:
%%R
t_test_results = t.test(uk_scores, us_scores)
print(t_test_results)


`t_test_results` is a list with different elements which provide information on the t-test we have carried out. The `print()` function gives us a summary of the test with the following information:

- t-statisic
- degrees of freedom
- p-value (although in this case it just indicates it's below $2.2\times 10^{-16}$)
- 95% CI on the "true" group difference

If we want to save the p-value in a variable, we have to retrieve it from the list.


In [ ]:
%%R
pval_uk_us = t_test_results$p.value # or t_test_results[["p.value"]]
print(paste("The p-value is:", pval_uk_us))


### Paired observations?

In our case, each observation in the data frame is independent from each other, but it is common in biomedical contexts to find measurements that come from the same sample (the same patient before and after administrating a drug).
In such cases, we say that the data is **paired**, and it means that the observations are likely correlated because they come from the same sample.
A special test exists to account for that, and it can be performed with the same `t.test` function.

Run `help(t.test)` to find out which parameter you'd have to change if you had paired samples.

In [ ]:
%%R

# Your code here

## Anova - extending t-tests

In [ ]:
%%R
uk_scores = movies$score[movies$country=="UK"]
us_scores = movies$score[movies$country=="USA"]
france_scores = movies$score[movies$country=="France"]
group_means = tibble(country=c("UK","USA", "France"), grpmean=c(mean(uk_scores), mean(us_scores), mean(france_scores)))

ggplot(movies, aes(x=score, color=country, fill = country)) +
  geom_density(data=movies %>% filter(country == "USA"),alpha=0.4) +
  geom_density(data=movies %>% filter(country == "UK"),alpha=0.4) +
  geom_density(data=movies %>% filter(country == "France"),alpha=0.4) +
  geom_vline(data = group_means, aes(xintercept = grpmean, color=country), size=1, linetype=2) + 
  #annotate("segment", x=min(group_means$grpmean), xend=max(group_means$grpmean), y=0.55, yend = 0.55, color="black") + 
  labs(title = "ANOVA: extending t-tests",
       subtitle = "Here we test whether the variance between groups is larger than the variance within groups")

Another, more proper way of seeing it is:

In [ ]:
%%R

uk_scores = movies$score[movies$country=="UK"]
us_scores = movies$score[movies$country=="USA"]
france_scores = movies$score[movies$country=="France"]
group_means = tibble(country=c("UK","USA", "France"), grpmean=c(mean(uk_scores), mean(us_scores), mean(france_scores))) %>% mutate(country = factor(country, levels = c("UK", "USA", "France")))

movies %>% 
  filter(country %in% c("UK", "USA", "France")) %>% 
  mutate(country = factor(country, levels = c("UK", "USA", "France"))) %>% 
  ggplot(aes(x=country, y=score, color=country)) +
  geom_jitter(alpha=0.3) + 
  geom_segment(data=group_means, aes(x=as.numeric(country)-0.5, xend=as.numeric(country)+0.5, y=grpmean, yend = grpmean), color="black") +
  annotate("segment", x=0, xend=4, y=mean(group_means$grpmean), yend=mean(group_means$grpmean), linetype=2) + 
  labs(title = "ANOVA from another perspective",
       subtitle = "The goal here is to determine whether the variance between the groups\nislarger than the variance within them.")



If the variance within each group is small and the variance between them is large, then we could say that there is a difference in the means of the groups, because each distribution is separated from the others.

### One way

The syntax to make an ANOVA test is slightly more complicated, but not too much. Instead of just providing two vectors, the input will be the entire data frame (the `data` argument).
Once we have that, we are going to tell R that we want to model our outcome variable (scores, `y`) as a function of a predictive variable (country, `x`). For that, we use the formula notation:

`y ~ x`

and we give it as first argument to the `aov()` function (**a**nalysis **o**f **v**ariance)


In [ ]:
%%R

minidf = filter(movies, country %in% c("UK", "USA", "France"))
anova_test = aov(score ~ country, data=minidf)
print(summary(anova_test))



It seems like there is a significant difference between the groups!. As opposed to the t.test, where we had an estimate for the difference in means (the effect size), here we do not obtain anything of the sort, since the question we are asking is "Is there a difference at all between the groups?" and not "Which groups are different to which?".

To answer the latter question, we would have to do pairwise comparison between all the categories tested, which would imply a need to do multiple comparisons correction.

### Two-way

ANOVA can be extended to measure the effects of several categorical variables together. In our case, we could be looking at the differences between both countries and genres at the same time. Such a test is called two-way ANOVA and is run with the same `aov()` function.

The thing that changes is that now we add another term to the formula:

`y ~ x1*x2`

The "*" indicates that we also want to see if there is an interaction between both grouping variables.


In [ ]:
%%R
minidf = filter(movies, country %in% c("UK", "USA", "France"), genre %in% c("Action", "Horror", "Drama"))
two_way_anova_test = aov(score ~ country*genre, data=minidf)
print(summary(two_way_anova_test))


## Non-parametric equivalents

### Wilcoxon rank-sum (Mann-Whitney U)

- t-test assumes normality for each group
  - Only if each group is normal the mean and stdev are proper representations of the true expectation and variance of the population
- Data might not be normally distributed!
  - Non-parametric tests to the rescue
  - Less powerful when data **is** normally distrib.

In [ ]:
%%R
uk_scores = movies$score[movies$country=="UK"]
us_scores = movies$score[movies$country=="USA"]

wilcx_test = wilcox.test(uk_scores, us_scores)
print(wilcx_test)


### Wilcoxon signed rank

Guess what this is equivalent to :D

### Kruskal-Wallis test

If we want to extend the Wilcoxon test to accept more than 2 groups (just like ANOVA), we can use the Kruskal-Wallis test:

In [ ]:
%%R

minidf = filter(movies, country %in% c("UK", "USA", "France"))
krskl_test = kruskal.test(score ~ country, data=minidf)
print((krskl_test))


## Further reading

[This blog post](http://www.rebeccabarter.com/blog/2017-02-17-anova/) has helped me a lot in understanding ANOVA better.